<a href="https://colab.research.google.com/github/ethanduncan65/NFL-Team-Analyzer/blob/main/NFL_Team_Analyzer_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fantasy Football FLEX Production Analyzer 1.0 - Researching Capabilites of ESPN Data

In [ ]:
from bs4 import BeautifulSoup as soup  
from urllib.request import urlopen as uReq  
import pandas as pd

numWeeks = 3

In [ ]:
df_player = pd.DataFrame(columns = ['Week', 'Team', 'Opp', 'Tgt', 'Rec', 'Pass Yds', 'Pass TD', 'Car', 'Rush Yds', 'Rush TD', 'Points', '% Share'],  index = range(0, numWeeks))

player_page_url = 'https://www.espn.com/nfl/player/gamelog/_/id/4047650/dk-metcalf'
# ...
player_uClient = uReq(player_page_url)
# ...
player_page_soup = soup(player_uClient.read(), "html.parser")
player_uClient.close()

#player_stats = player_page_soup.find('tr', {'class': "Table__TR Table__TR--sm Table__even"})
#player_statsX = player_page_soup.findAll('td', {'class': "Table__TD"})
#player_statsX[0].get_text()
#player_statsX[4].get_text()
#player_statsX[36].get_text()



# determine if player is RB or WR/TE
player_stats = player_page_soup.findAll('td', {'class': "Table__TD"})
player_position = player_page_soup.findAll('th', {'class': "tc Table__TH"})
player_position_str = player_position[0].get_text()
if player_position_str=='Rushing':
  #
  k=0
  for r in range(0, numWeeks):
    # add week, team, opp
    df_player.at[r, 'Tgt'] = player_stats[9+k].get_text()
    df_player.at[r, 'Rec'] = player_stats[8+k].get_text()
    df_player.at[r, 'Pass Yds'] = player_stats[10+k].get_text()
    df_player.at[r, 'Pass TD'] = player_stats[9+k].get_text()
    df_player.at[r, 'Car'] = player_stats[3+k].get_text()
    df_player.at[r, 'Rush Yds'] = player_stats[4+k].get_text()
    df_player.at[r, 'Rush TD'] = player_stats[6+k].get_text()
    k+=18
else:
  #
  k=0
  for r in range(0, numWeeks):
    # add week, team, opp
    df_player.at[r, 'Tgt'] = player_stats[4+k].get_text()
    df_player.at[r, 'Rec'] = player_stats[3+k].get_text()
    df_player.at[r, 'Pass Yds'] = player_stats[5+k].get_text()
    df_player.at[r, 'Pass TD'] = player_stats[7+k].get_text()
    df_player.at[r, 'Car'] = player_stats[9+k].get_text()
    df_player.at[r, 'Rush Yds'] = player_stats[10+k].get_text()
    df_player.at[r, 'Rush TD'] = player_stats[13+k].get_text()
    k+=18

# reverse order of df_player to align with format of df's below

In [ ]:
Team = "Seattle Seahawks" #@param ["Seattle Seahawks", "San Francisco 49ers"]
vsTeam = "San Francisco 49ers" #@param ["Seattle Seahawks", "San Francisco 49ers"]
Format = "Standard" #@param ["Standard", "Half-PPR", "Full-PPR"]
team_sched_dict = {"Seattle Seahawks": [401326318, 401326360, 401326376],
                   "San Francisco 49ers": [401326317, 401326355, 401326377]}
game_ids = team_sched_dict.get(Team)
vs_game_ids = team_sched_dict.get(vsTeam)

team_name_dict = {"Seattle Seahawks": 'SEA',
                  "San Francisco 49ers": 'SF'}
team_abbrev = team_name_dict.get(Team)
vs_team_abbrev = team_name_dict.get(vsTeam)

In [ ]:
df = pd.DataFrame(columns = ['Week', 'Team', 'Opp', 'Tgt', 'Rec', 'Pass Yds', 'Pass TD', 'Car', 'Rush Yds', 'Rush TD', 'Points'],  index = range(0, numWeeks))

for i in range(0, numWeeks):
  page_url = 'https://www.espn.com/nfl/boxscore/_/gameId/' + str(game_ids[i]) + ''
  # ...
  uClient = uReq(page_url)
  # ...
  page_soup = soup(uClient.read(), "html.parser")
  uClient.close()

  df.at[i, 'Week'] = i+1

  # get team name
  away_team = page_soup.find('div', {'class': "team away"})
  away_team_name = away_team.find('span', {'class': "abbrev"})
  away_team_name_str = away_team_name.get_text()

  # get home name
  home_team = page_soup.find('div', {'class': "team home"})
  home_team_name = home_team.find('span', {'class': "abbrev"})
  home_team_name_str = home_team_name.get_text()

  # determine if team is home or away 
  is_home_team = (team_abbrev==home_team_name_str)

  # scrape away or home team data based on if team is away or home
  if is_home_team:
    # add Team and Opp to df
    df.at[i, 'Team'] = home_team_name_str
    df.at[i, 'Opp'] = away_team_name_str
    # add Tgt, Rec, Yds, and TD to df
    away_boxscore_div = page_soup.findAll('div', {'class': "col column-two gamepackage-home-wrap"})
    pass_data = away_boxscore_div[2].find('tr', {'class': "highlight"})
    df.at[i, 'Tgt'] = pass_data.find('td', {'class': "tgts"}).get_text()
    df.at[i, 'Rec'] = pass_data.find('td', {'class': "rec"}).get_text()
    df.at[i, 'Pass Yds'] = pass_data.find('td', {'class': "yds"}).get_text()
    df.at[i, 'Pass TD'] = pass_data.find('td', {'class': "td"}).get_text()
    rush_data = away_boxscore_div[1].find('tr', {'class': "highlight"})
    df.at[i, 'Car'] = rush_data.find('td', {'class': "car"}).get_text()
    df.at[i, 'Rush Yds'] = rush_data.find('td', {'class': "yds"}).get_text()
    df.at[i, 'Rush TD'] = rush_data.find('td', {'class': "td"}).get_text()

  else:
    # add Team and Opp to df
    df.at[i, 'Team'] = away_team_name_str
    df.at[i, 'Opp'] = home_team_name_str
    # add Tgt, Rec, Yds, and TD to df
    away_boxscore_div = page_soup.findAll('div', {'class': "col column-one gamepackage-away-wrap"})
    pass_data = away_boxscore_div[2].find('tr', {'class': "highlight"})
    df.at[i, 'Tgt'] = pass_data.find('td', {'class': "tgts"}).get_text()
    df.at[i, 'Rec'] = pass_data.find('td', {'class': "rec"}).get_text()
    df.at[i, 'Pass Yds'] = pass_data.find('td', {'class': "yds"}).get_text()
    df.at[i, 'Pass TD'] = pass_data.find('td', {'class': "td"}).get_text()
    rush_data = away_boxscore_div[1].find('tr', {'class': "highlight"})
    df.at[i, 'Car'] = rush_data.find('td', {'class': "car"}).get_text()
    df.at[i, 'Rush Yds'] = rush_data.find('td', {'class': "yds"}).get_text()
    df.at[i, 'Rush TD'] = rush_data.find('td', {'class': "td"}).get_text()




In [ ]:
df2 = pd.DataFrame(columns = ['Week', 'Team', 'Opp', 'Tgt', 'Rec', 'Pass Yds', 'Pass TD', 'Car', 'Rush Yds', 'Rush TD', 'Points'],  index = range(0, numWeeks))

for i in range(0, numWeeks):
  vs_page_url = 'https://www.espn.com/nfl/boxscore/_/gameId/' + str(vs_game_ids[i]) + ''
  # ...
  vs_uClient = uReq(vs_page_url)
  # ...
  vs_page_soup = soup(vs_uClient.read(), "html.parser")
  vs_uClient.close()

  df2.at[i, 'Week'] = i+1

  # get team name
  away_team = vs_page_soup.find('div', {'class': "team away"})
  away_team_name = away_team.find('span', {'class': "abbrev"})
  away_team_name_str = away_team_name.get_text()

  # get home name
  home_team = vs_page_soup.find('div', {'class': "team home"})
  home_team_name = home_team.find('span', {'class': "abbrev"})
  home_team_name_str = home_team_name.get_text()

  # determine if team is home or away 
  is_home_team = (vs_team_abbrev!=home_team_name_str)

  # scrape away or home team data based on if team is away or home
  if is_home_team:
    # add Team and Opp to df
    df2.at[i, 'Team'] = home_team_name_str
    df2.at[i, 'Opp'] = away_team_name_str
    # add Tgt, Rec, Yds, and TD to df
    away_boxscore_div = vs_page_soup.findAll('div', {'class': "col column-two gamepackage-home-wrap"})
    pass_data = away_boxscore_div[2].find('tr', {'class': "highlight"})
    df2.at[i, 'Tgt'] = pass_data.find('td', {'class': "tgts"}).get_text()
    df2.at[i, 'Rec'] = pass_data.find('td', {'class': "rec"}).get_text()
    df2.at[i, 'Pass Yds'] = pass_data.find('td', {'class': "yds"}).get_text()
    df2.at[i, 'Pass TD'] = pass_data.find('td', {'class': "td"}).get_text()
    rush_data = away_boxscore_div[1].find('tr', {'class': "highlight"})
    df2.at[i, 'Car'] = rush_data.find('td', {'class': "car"}).get_text()
    df2.at[i, 'Rush Yds'] = rush_data.find('td', {'class': "yds"}).get_text()
    df2.at[i, 'Rush TD'] = rush_data.find('td', {'class': "td"}).get_text()


  else:
    # add Opp to df
    df2.at[i, 'Team'] = away_team_name_str
    df2.at[i, 'Opp'] = home_team_name_str
    # add Tgt, Rec, Yds, and TD to df
    away_boxscore_div = vs_page_soup.findAll('div', {'class': "col column-one gamepackage-away-wrap"})
    pass_data = away_boxscore_div[2].find('tr', {'class': "highlight"})
    df2.at[i, 'Tgt'] = pass_data.find('td', {'class': "tgts"}).get_text()
    df2.at[i, 'Rec'] = pass_data.find('td', {'class': "rec"}).get_text()
    df2.at[i, 'Pass Yds'] = pass_data.find('td', {'class': "yds"}).get_text()
    df2.at[i, 'Pass TD'] = pass_data.find('td', {'class': "td"}).get_text()
    rush_data = away_boxscore_div[1].find('tr', {'class': "highlight"})
    df2.at[i, 'Car'] = rush_data.find('td', {'class': "car"}).get_text()
    df2.at[i, 'Rush Yds'] = rush_data.find('td', {'class': "yds"}).get_text()
    df2.at[i, 'Rush TD'] = rush_data.find('td', {'class': "td"}).get_text()

In [ ]:
df

In [ ]:
df2

In [ ]:
# div containing the entire boxscore passing section
boxscore_div = page_soup.find('div', {'class': "gamepackage-passing"})



In [ ]:
# div containing the entire boxscore passing section
pass_boxscore_div = page_soup.find('div', {'class': "gamepackage-passing"})
pass_boxscore_div.find('td', {'class': "c-att"})

# away team boxscore passing section
away_boxscore_div = page_soup.findAll('div', {'class': "col column-one gamepackage-away-wrap"})
pass_data = away_boxscore_div[1].find('tr', {'class': "highlight"})
passRatio = pass_data.find('td', {'class': "car"}).get_text()

passRatio.split('/')[1]

In [ ]:
# get comp/att for away QB
test_div = page_soup.find('td', {'class': "team-name"})
test_div.get_text()